In [228]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.max_columns=None
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from dateutil.relativedelta import *
from dateutil.rrule import *
from datetime import datetime as dt

%config InteractiveShell.ast_node_interactivity = 'all'
#import dependencies

In [229]:
df = pd.read_parquet("../Proyecto-Taxis-NYC/data/yellow_t_2018-01.parquet", engine="pyarrow")

In [230]:
def t_datime(df):
    df["dayofmonth"] = df['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day
    df["hora"]= df.tpep_pickup_datetime.dt.floor("H")

    df.drop(columns=["congestion_surcharge","airport_fee","store_and_fwd_flag"], inplace=True)
    return df

def t_int(df):
    df['passenger_count'] = pd.to_numeric(df['passenger_count'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    df['PULocationID'] = pd.to_numeric(df['PULocationID'], downcast="integer" )
    df['DOLocationID'] = pd.to_numeric(df['DOLocationID'], downcast="integer" )
    df['payment_type'] = pd.to_numeric(df['payment_type'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    return df

def t_float(df):
    df['trip_distance'] = pd.to_numeric(df['trip_distance'], downcast="float" )
    df['fare_amount'] = pd.to_numeric(df['fare_amount'], downcast="float" )
    df['extra'] = pd.to_numeric(df['extra'], downcast="float" )
    df['mta_tax'] = pd.to_numeric(df['mta_tax'], downcast="float" )
    df['tip_amount'] = pd.to_numeric(df['tip_amount'], downcast="float" )
    df['tolls_amount'] = pd.to_numeric(df['tolls_amount'], downcast="float" )
    df['improvement_surcharge'] = pd.to_numeric(df['improvement_surcharge'], downcast="float" )
    df['total_amount'] = pd.to_numeric(round(df['total_amount'] ))
    return df

def total_transform(df):
    df = t_datime(df)
    df = t_int(df)
    df = t_float(df)
    return df

df =  total_transform(df)

In [231]:
df_Impar = df[df.dayofmonth  == 3]

In [232]:
def sorted(df):
    global bol
    bol = 1
    df.sort_values(by='tpep_pickup_datetime', ascending=True, inplace=True)
    df = df[df["tpep_pickup_datetime"] >= '2018-01-01 00:00:00']
    if bol == 1:
        df = df[df["tpep_pickup_datetime"] <= '2018-02-01 00:00:00']
        bol = 0
    return df

df_Impar = sorted(df_Impar)

In [233]:
def boroughs (df_Impar):
    #se carga el csv con las zonas
    zona = pd.read_csv("taxi+_zone_lookup.csv")
    #se borran las columnas que no usamos
    zona.drop(columns=["Zone","service_zone"], inplace=True)
    #cambiamos el nombre para poder hacer el merge
    zona.rename(columns={"LocationID": "PULocationID"},inplace=True)
    #se hace el merge y se crea la columnna resultante del merge "Borough"
    df_Impar = pd.merge(df_Impar,zona, how="left", on=["PULocationID"])
    #se le cambia el nombre a "boroug" para que haga referencia al borough de partida
    df_Impar.rename(columns={"Borough": "PUBorough"},inplace=True)
    #mismo procedimiento para borough de llegada
    zona.rename(columns={"PULocationID": "DOLocationID"},inplace=True)
    df_Impar = pd.merge(df_Impar,zona, how="left", on=["DOLocationID"])
    df_Impar.rename(columns={"Borough": "DOLocation"},inplace=True)
    return df_Impar

df_Impar = boroughs(df_Impar)

In [234]:
#documentacion mismo nombre del clima en cada update

clima = pd.read_csv("C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/clima.csv")
clima.drop(columns=["name","dew","humidity","precipprob","preciptype","severerisk","uvindex","icon","stations"], inplace=True)
clima.drop(columns=["windgust","windspeed","winddir","sealevelpressure","cloudcover","visibility","solarradiation","solarenergy"], inplace=True)
clima["datetime"]=[x.replace('T'," ") for x in clima["datetime"]]
clima.datetime = clima['datetime'].apply(pd.to_datetime)

In [235]:
def t_clima():
    clima['temp'] = pd.to_numeric(clima['temp'], downcast="float" )
    clima['feelslike'] = pd.to_numeric(round(clima['feelslike'] ))
    clima['precip'] = pd.to_numeric(clima['precip'], downcast="float" )
    clima['snow'] = pd.to_numeric(clima['snow'], downcast="float" )
    clima['snowdepth'] = pd.to_numeric(clima['snowdepth'], downcast="float" )
    clima["conditions"]=clima["conditions"].astype(str)
    return clima

clima = t_clima()

In [236]:
def control_clima(dat,clima):
    #aplicar sort fecha
    clima.sort_values(by='datetime', ascending=True, inplace=True)
    date = dat
    if (clima["datetime"] >= date).any():
        clima = clima[clima["datetime"] >= date]
        date_next= date + relativedelta(months=+1)
        clima = clima[clima["datetime"] <= date_next]
        return clima
    else:
        date+= relativedelta(months=+1)
        return control_clima(dat)

clima = control_clima(dt(2018,1,1),clima)

In [237]:
#clima in global

def aux ():
    
    clima.rename(columns={"datetime": "hora"},inplace=True)
    return clima

clima = aux()

In [238]:
#df_Impar = pd.merge(df_Impar,clima, how="left", on=["hora"])

In [239]:
def neg_values(df_Impar):   
    df_Impar = df_Impar.drop(df_Impar[df_Impar.fare_amount<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.trip_distance<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.total_amount<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.RatecodeID==6].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.trip_distance==0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.fare_amount==0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.total_amount==0].index)
    return df_Impar

df_Impar=neg_values(df_Impar)

In [240]:
#Tratamiento de Outliers columna trip_distance
Q1 = df_Impar["trip_distance"].quantile(0.01)
Q3 = df_Impar["trip_distance"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_Sup_trip = (Q3 + (1.5*IQR)) 
#se crea la columna marcador
df_Impar["trip_distance_out"] = df_Impar.apply(lambda row: 1 if row["trip_distance"]>outliers_Sup_trip else 0, axis=1)

#se marco trip distance, ahora vamos por fare amount

#Tratamiento de Outliers columna fare_amount
Q1 = df_Impar["fare_amount"].quantile(0.01)
Q3 = df_Impar["fare_amount"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_Sup_fare = (Q3 + (1.5*IQR)) 
#se marco fare amount
df_Impar["fare_amount_out"] = df_Impar.apply(lambda row: 1 if row["fare_amount"]>outliers_Sup_fare else 0, axis=1)

#Tratamiento de Outliers columna tip_amount
Q1 = df_Impar["tip_amount"].quantile(0.01)
Q3 = df_Impar["tip_amount"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_Sup_tip = (Q3 + (1.5*IQR)) 
df_Impar["tip_amount_out"] = df_Impar.apply(lambda row: 1 if row["tip_amount"]>outliers_Sup_tip else 0, axis=1)

#se marco tip amount, ahora le toca a total amount

#Tratamiento de Outliers columna total_amount
Q1 = df_Impar["total_amount"].quantile(0.01)
Q3 = df_Impar["total_amount"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_Sup_total_a = (Q3 + (1.5*IQR)) 
df_Impar["total_amount_out"] = df_Impar.apply(lambda row: 1 if row["tip_amount"]>outliers_Sup_total_a else 0, axis=1)

In [241]:
df_Impar["outlier"] = df_Impar.apply(lambda row: 1 if row["trip_distance_out"]==1 or row["fare_amount_out"]==1 or
row["total_amount_out"]==1 or row["tip_amount_out"]==1 else 0, axis=1)

In [242]:
df_Impar.to_parquet('C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/data/df_Impar_marcado.parquet',engine="pyarrow")

In [243]:
df_Impar = pd.read_parquet("C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/data/df_Impar_marcado.parquet", engine="pyarrow")

Arrancamos con SQL

In [244]:
import mysql.connector as msql
from mysql.connector import Error
from sqlalchemy import create_engine
from multiprocessing import connection

In [245]:
#df total de los taxis
df = pd.read_parquet("C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/data/df_Impar_marcado.parquet", engine="pyarrow")

In [246]:
df = df[df.dayofmonth == 3]

In [247]:
#buscar los df ya tratados
df_Zona = pd.read_csv("taxi+_zone_lookup.csv", sep=",")
#df_Clima=pd.read_csv("Proyecto-Taxis-NYC/clima.csv", sep=',')

In [248]:
#CREA BASE DE DATOS
try:
    conn = msql.connect(host='127.0.0.1', user='root',  
                        password='Andres1')#give ur username, password
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS taxi_1;")
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Database is created


In [249]:
host_name,db_name, u_name, u_pass, port_num  = "127.0.0.1","taxi_1", "root", "Andres1", "3306"

In [250]:
try:
    connection = mysql.connector.connect(
    host= host_name,
    user= u_name,
    password= u_pass,
    database= db_name
    )
  
    if connection.is_connected():
        print('Conexion exitosa')
        info_server = connection.get_server_info()
        print(info_server)
except Exception as ex:
    print(ex)

name 'mysql' is not defined


In [251]:
#Creacion del motor de base de datos
engine = create_engine("mysql+mysqlconnector://" + u_name + ":" + u_pass + "@" 
                        + host_name + ":" + port_num + "/" + db_name, echo=False)

In [252]:
#Importacion del dataframe a nuestra base de datos en SQL 
#CAMBIAR LOS NOMBRES DEL DF QUE QUEREMOS CARGAR Y EL NOMBRE QUE QUEREMOS QUE TENGA LA TABLA EN SQL
#df.to_sql(name="taxis", con=engine, chunksize=200, if_exists="append", index=False)

In [253]:
df_Zona.to_sql(name="zona", con=engine, if_exists="append", index=False)

265

In [254]:
clima.to_sql(name="clima", con=engine, if_exists="append", index=False)

745

creacion de tablas para sql

creacion Clima_conditions

In [255]:
Clima_conditions = clima.conditions.unique()
Clima_conditions = pd.DataFrame(Clima_conditions)
Clima_conditions["conditions"] = Clima_conditions
Clima_conditions = Clima_conditions.drop(columns=0)
Clima_conditions.reset_index(inplace=True)
Clima_conditions.rename(columns={"index":"IdCondiciones"},inplace=True)

In [256]:
Clima_conditions.to_sql(name="clima_conditions", con=engine, if_exists="append", index=False)

8

creacion tabla Borough

In [257]:
Borough = pd.read_csv("Taxi+_zone_lookup.csv", usecols=['Borough'])
Borough = Borough.Borough.unique()
Borough = pd.DataFrame(Borough)
Borough["Borough"] = Borough
Borough = Borough.drop(columns=0)
Borough.reset_index(inplace=True)
Borough.rename(columns={"index":"IdBorough"},inplace=True)

In [258]:
Borough.to_sql(name="borough", con=engine, if_exists="append", index=False)

7

creacion tabla Vendor

In [259]:
Vendor = df_Impar.VendorID.unique()
Vendor = pd.DataFrame(Vendor)
Vendor["VendorID"] = Vendor
Vendor = Vendor.drop(columns=0)
Vendor["Vendor"] = Vendor.VendorID
Vendor.Vendor.replace(1,"Tecnologías móviles creativas", inplace=True)
Vendor.Vendor.replace(2,"VeriFone Inc.", inplace=True)


In [260]:
Vendor.to_sql(name="vendor", con=engine, if_exists="append", index=False)

2

creacion tabla Ratecode

In [262]:
RatecodeID = df_Impar.RatecodeID.unique()
RatecodeID = pd.DataFrame(RatecodeID)
RatecodeID["RatecodeID"] = RatecodeID
RatecodeID = RatecodeID.drop(columns=0)
RatecodeID["Ratecode"] = RatecodeID.RatecodeID
RatecodeID.Ratecode.replace(1,"Tarifa estándar", inplace=True)
RatecodeID.Ratecode.replace(2,"JFK", inplace=True)
RatecodeID.Ratecode.replace(3,"Nueva York", inplace=True)
RatecodeID.Ratecode.replace(4,"Nasáu o Westchester", inplace=True)
RatecodeID.Ratecode.replace(5,"Tarifa negociada", inplace=True)



In [263]:
RatecodeID.to_sql(name="ratecode", con=engine, if_exists="append", index=False)

5

eliminacion de columnas redundantes de df_impar

tenemos que borrar PULocationID y DOLocationID y reemplazar PUBorough y DOLocation por sus ids.

In [264]:
df_Impar['PUBorough']=np.where(df_Impar['PUBorough']=="EWR",0,df_Impar['PUBorough'])
df_Impar['PUBorough']=np.where(df_Impar['PUBorough']=="Queens",1,df_Impar['PUBorough'])
df_Impar['PUBorough']=np.where(df_Impar['PUBorough']=="Bronx",2,df_Impar['PUBorough'])
df_Impar['PUBorough']=np.where(df_Impar['PUBorough']=="Manhattan",3,df_Impar['PUBorough'])
df_Impar['PUBorough']=np.where(df_Impar['PUBorough']=="Staten Island",4,df_Impar['PUBorough'])
df_Impar['PUBorough']=np.where(df_Impar['PUBorough']=="Brooklyn",5,df_Impar['PUBorough'])
df_Impar['PUBorough']=np.where(df_Impar['PUBorough']=="Unknown",6,df_Impar['PUBorough'])

df_Impar.rename(columns={"DOLocation":"DOBorough"},inplace=True)
df_Impar['DOBorough']=np.where(df_Impar['DOBorough']=="EWR",0,df_Impar['DOBorough'])
df_Impar['DOBorough']=np.where(df_Impar['DOBorough']=="Queens",1,df_Impar['DOBorough'])
df_Impar['DOBorough']=np.where(df_Impar['DOBorough']=="Bronx",2,df_Impar['DOBorough'])
df_Impar['DOBorough']=np.where(df_Impar['DOBorough']=="Manhattan",3,df_Impar['DOBorough'])
df_Impar['DOBorough']=np.where(df_Impar['DOBorough']=="Staten Island",4,df_Impar['DOBorough'])
df_Impar['DOBorough']=np.where(df_Impar['DOBorough']=="Brooklyn",5,df_Impar['DOBorough'])
df_Impar['DOBorough']=np.where(df_Impar['DOBorough']=="Unknown",6,df_Impar['DOBorough'])


In [265]:
df_Impar.drop(columns={"PULocationID","DOLocationID"},inplace=True)

In [266]:
df_Impar.to_sql(name="taxis", con=engine, chunksize=200, if_exists="append", index=False)

265213

creacion de tabla PaymentType

In [276]:
PaymentType = df_Impar.payment_type.unique()
PaymentType = pd.DataFrame(PaymentType)
PaymentType["PaymentTypeID"] = PaymentType
PaymentType = PaymentType.drop(columns=0)
PaymentType["Payment"] = PaymentType.PaymentTypeID
PaymentType.Payment.replace(1,"tarjeta de crédito", inplace=True)
PaymentType.Payment.replace(2,"Efectivo", inplace=True)
PaymentType.Payment.replace(3,"Sin cargo", inplace=True)
PaymentType.Payment.replace(4,"Disputa", inplace=True)


In [278]:
PaymentType.to_sql(name="payment_type", con=engine, chunksize=200, if_exists="append", index=False)

4